In [22]:
#Importing the necessary libraries
import pandas as pd
!pip install openai
!pip install --upgrade openai
import openai
import re
import random
import requests
import json

In [2]:
#Importing the dataset 1
df1 = pd.read_csv('/content/Mental_Health_FAQ.csv')

In [3]:
#Size of the dataset 1
df1.shape

(98, 3)

In [4]:
#Columns of the dataset 1
df1.columns

Index(['Question_ID', 'Questions', 'Answers'], dtype='object')

In [5]:
#Dropping the Question ID Column from dataset 1
df1.drop('Question_ID', axis=1, inplace=True)

In [6]:
#Checking for null values in dataset 1
df1.isnull().sum()

,0
Questions,0
Answers,0


In [7]:
#Importing the dataset 2
df2 = pd.read_csv('/content/train.csv')

In [8]:
#Size of the dataset 2
df2.shape

(172, 1)

In [9]:
#Columns of the dataset 2
df2.columns

Index(['text'], dtype='object')

In [10]:
#Checking for null values in dataset 2
df2.isnull().sum()

,0
text,0


In [11]:
#Seperating the texts from the dataset 2 as Questions and Answers

def split_qa(row):
    human, assistant = row.split("<ASSISTANT>:")
    question = human.replace("<HUMAN>:", "").strip()
    answer = assistant.strip() if assistant else ""
    return pd.Series([question, answer])

df2[['Questions', 'Answers']] = df2['text'].apply(split_qa)


In [12]:
#Dropping the text column from dataset 2
df2.drop('text', axis=1, inplace=True)

In [13]:
#Size of the dataset 2
df2.shape

(172, 2)

In [14]:
#Columns of the dataset 2
df2.columns

Index(['Questions', 'Answers'], dtype='object')

In [15]:
#Merging the datasets
MHS_data = pd.concat([df1, df2], ignore_index=True)

In [16]:
#Size of the final dataset
MHS_data.shape

(270, 2)

In [17]:
#Columns of the final dataset
MHS_data.columns

Index(['Questions', 'Answers'], dtype='object')

In [18]:
#Checking for null values
MHS_data.isnull().sum()

,0
Questions,0
Answers,0


In [20]:
#Sample data
MHS_data.head()

,Questions,Answers
0,What does it mean to have a mental illness?,Mental illnesses are health conditions that di...
1,Who does mental illness affect?,It is estimated that mental illness affects 1 ...
2,What causes mental illness?,It is estimated that mental illness affects 1 ...
3,What are some of the warning signs of mental i...,Symptoms of mental health disorders vary depen...
4,Can people with mental illness recover?,"When healing from mental illness, early identi..."


In [ ]:
# Before going to the chatbot process, lets get the API key for OpenAI
file_path = '/content/GRS_API_Key.txt'

# Read the API key from the file
with open(file_path, 'r') as file:
    openai.api_key = file.read().strip()

print(f"OpenAI API Key: {openai.api_key}")

In [ ]:
# Before going to the chatbot process, lets get the API key for TextRazor
file_path = '/content/TR_API_Key.txt'

# Read the API key from the file
with open(file_path, 'r') as file:
    tr_api_key = file.read().strip()

print(f"TextRazor API Key: {tr_api_key}")

In [24]:
#Downloading the final dataset
MHS_data.to_csv('MHS_data.csv', index=False)

In [25]:
#Preparing the dataset for fine tuning
system_content = (
    "You are a highly trained and compassionate mental health advisor with expertise in providing empathetic and supportive responses to individuals seeking help. "
    "Your primary objective is to foster a safe and understanding environment for users to express their feelings, concerns, and questions related to mental health. "
    "Approach each conversation with genuine kindness, active listening, and a non-judgmental attitude, ensuring that users feel heard and validated in their experiences. "
    "In situations where users exhibit extreme emotions, such as overwhelming sadness, anxiety, or despair, respond with compassion and offer appropriate coping strategies. "
    "Suggest grounding techniques, mindfulness practices, or breathing exercises that can help alleviate their distress. If a user articulates thoughts of self-harm or expresses feelings of hopelessness, prioritize their safety by encouraging them to talk to someone who can provide immediate support, such as a mental health professional or a crisis hotline. "
    "When users appear to be in significant emotional distress or express feelings of isolation, gently encourage them to consider connecting with live support services or professionals who can offer further assistance. Always frame this suggestion as a positive step towards their well-being. "
    "Make sure to respond to each inquiry with relevant and accurate information, guiding users toward understanding their feelings and circumstances without bias or shame. Provide educational resources, coping mechanisms, and tools that empower users to manage their mental health effectively. "
    "Throughout the conversation, remain adaptable to the user’s needs, and tailor your responses based on their emotional state. Be mindful of the words you choose, ensuring they convey empathy and understanding. Always invite users to share more about their feelings and experiences, fostering a deeper connection and support. "
    "End each interaction by checking in with users, asking if they feel satisfied with the conversation, and offering additional resources or suggestions for their mental health needs. Emphasize the importance of seeking help when needed and reassure them that they are not alone in their journey."
)

# Create a JSONL file with the required format
jsonl_file_path = '/content/mental_health_faqs.jsonl'
with open(jsonl_file_path, 'w') as jsonl_file:
    for index, faq in MHS_data.iterrows():
        json_entry = {
            "messages": [
                {"role": "system", "content": system_content},
                {"role": "user", "content": faq["Questions"]},
                {"role": "assistant", "content": faq["Answers"]}
            ]
        }
        json_string = json.dumps(json_entry)
        jsonl_file.write(f"{json_string}\n")


In [ ]:
#Generating File
client = openai.OpenAI(api_key=openai.api_key)
with open('/content/mental_health_faqs.jsonl', 'rb') as file:
    file_response = client.files.create(
        file=file,
        purpose='fine-tune'
    )
file_id = file_response.id
print(f'File ID: {file_id}')

In [27]:
#Fine tune response
fine_tune_response  = client.fine_tuning.jobs.create(
    training_file=file_id,
    model="gpt-4o-mini-2024-07-18"
)

In [ ]:
#Fine tune response object
fine_tune_response

In [ ]:
#Getting job id
fine_tune_id = fine_tune_response.id
print(f'Fine-tune ID: {fine_tune_id}')

In [ ]:
#Monitor Fine Tuning
response = client.fine_tuning.jobs.retrieve(fine_tune_id)
print(response)

In [ ]:
#Getting the fine tune model id
fine_tuned_model_id = response.fine_tuned_model
print(f'Fine-tuned model ID: {fine_tuned_model_id}')

In [45]:
# Define a list of keywords and phrases related to mental health FAQs
faq_keywords = {
    "anxiety": ["what is anxiety", "signs of anxiety", "how to deal with anxiety"],
    "stress": ["how to deal with stress", "stress management", "symptoms of stress"],
    "depression": ["what are the signs of depression", "how to cope with depression", "depression symptoms"],
}
#Categorise the user input
def is_faq_query(user_input):
    """Check if the user input matches any of the FAQ keywords or phrases."""
    user_input_lower = user_input.lower()
    for topic, phrases in faq_keywords.items():
        for phrase in phrases:
            if re.search(r'\b' + re.escape(phrase) + r'\b', user_input_lower):
                return True, topic
    return False, None

In [61]:
#Sentiment Analysis
def analyze_sentiment(user_input):
    # Text Razor API URL
    url = "https://api.textrazor.com/"

    # Headers for the request, including your API key
    headers = {
        'x-textrazor-key': tr_api_key,
        'Content-Type': 'application/json'
    }

    data = {
        'text': user_input,
        'extractors': 'sentiment'
    }

    # Make the POST request
    response = requests.post(url, data=data, headers=headers)

    # Parse the JSON response
    result = response.json()

    # Check if the request was successful and print the sentiment
    if 'response' in result and 'sentiment' in result['response'] and 'score' in result['response']['sentiment']:
        sentiment = result['response']['sentiment']['score']
        return sentiment
    else:
        return 10

In [63]:
#Suggest External Resources
def suggest_resources():
    """Provide mental health resources specific to India."""
    resources = [
        "Here are some resources you might find helpful:\n"
        "1. Vandrevala Foundation Helpline: 1860 266 2345 / 9999 666 555 - A confidential helpline for emotional support.",
        "2. AASRA: 91-22-2754 6669 - A 24-hour helpline for those in emotional distress.",
        "3. iCall: +91 9152987821 - Offers free mental health support and counseling.",
        "4. Snehi: +91 9582208181 - Provides emotional support for those in crisis.",
        "5. The National Institute of Mental Health and Neuro Sciences (NIMHANS): https://nimhans.ac.in/ - Provides mental health services and research."
        "Thank you for chatting! Take care."
    ]
    return "\n".join(resources)

In [ ]:
#Chatbot Implementation
def get_response(messages, use_faq_model=False):
    """Get response from OpenAI API using fine-tuned model or general model."""
    model = fine_tuned_model_id if use_faq_model else "gpt-4o-mini"

    url = "https://api.openai.com/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {openai.api_key}",
        "Content-Type": "application/json"
    }
    data = {
        "model": model,
        "messages": messages,
        "temperature": 0.7,
        "max_tokens": 150
    }

    try:
        response = requests.post(url, headers=headers, json=data)
        response.raise_for_status()
        completion = response.json()
        return completion['choices'][0]['message']['content']
    except requests.exceptions.RequestException:
        return "I'm experiencing technical difficulties at the moment. Please try again later."




# Define the initial prompt for the chatbot
initial_prompt = (
    "You are a highly trained and compassionate mental health advisor with expertise in providing empathetic and supportive responses to individuals seeking help. "
    "Your primary objective is to foster a safe and understanding environment for users to express their feelings, concerns, and questions related to mental health. "
    "Approach each conversation with genuine kindness, active listening, and a non-judgmental attitude, ensuring that users feel heard and validated in their experiences. "

    "In situations where users exhibit extreme emotions, such as overwhelming sadness, anxiety, or despair, respond with compassion and offer appropriate coping strategies. "
    "Suggest grounding techniques, mindfulness practices, or breathing exercises that can help alleviate their distress. If a user articulates thoughts of self-harm or expresses feelings of hopelessness, prioritize their safety by encouraging them to talk to someone who can provide immediate support, such as a mental health professional or a crisis hotline. "

    "When users appear to be in significant emotional distress or express feelings of isolation, gently encourage them to consider connecting with live support services or professionals who can offer further assistance. Always frame this suggestion as a positive step towards their well-being. "

    "Make sure to respond to each inquiry with relevant and accurate information, guiding users toward understanding their feelings and circumstances without bias or shame. Provide educational resources, coping mechanisms, and tools that empower users to manage their mental health effectively. "

    "Throughout the conversation, remain adaptable to the user’s needs, and tailor your responses based on their emotional state. Be mindful of the words you choose, ensuring they convey empathy and understanding. Always invite users to share more about their feelings and experiences, fostering a deeper connection and support. "

    "End each interaction by checking in with users, asking if they feel satisfied with the conversation, and offering additional resources or suggestions for their mental health needs. Emphasize the importance of seeking help when needed and reassure them that they are not alone in their journey."
)


# Main chatbot loop
messages = []
print("Hey! I am your Mental Health Advisor. You can ask me questions about mental health and share your feelings.")

# Add the initial prompt to the message history
messages.append({"role": "system", "content": initial_prompt})

while True:
    user_input = input("You: ")

    if user_input.lower() == 'exit':
        satisfied = input("Are you satisfied with the conversation? (yes/no): ")
        if satisfied.lower() == 'yes':
            additional_suggestion = input("Would you like me to suggest anything else? (yes/no): ")
            if additional_suggestion.lower() == 'no':
                print("Thank you for chatting! Take care.")
                break
            else:
                print("Sure, what else can I help you with?")
                continue
        else:
            print("I'm sorry to hear that. Please let me know how I can improve.")
            resources = suggest_resources()
            print(resources)
        break

    messages.append({"role": "user", "content": user_input})

    # Analyze sentiment to trigger specific responses
    sentiment_score = analyze_sentiment(user_input)

    if sentiment_score < -0.2:
        bot_response = (
            "I'm really sorry to hear that you're feeling this way. "
            "It's important to talk to someone who can help. Would you like to share with me?"
        )
        messages.append({"role": "assistant", "content": bot_response})
        print("Bot:", bot_response)
        continue

    is_faq, topic = is_faq_query(user_input)
    if is_faq:
        bot_response = get_response(messages, use_faq_model=True)
    else:
        bot_response = get_response(messages, use_faq_model=False)
    messages.append({"role": "assistant", "content": bot_response})
    print("Bot:", bot_response)







